## Лабораторная работа № 1 
## Выполнение разведочного анализа больших данных с использованием фреймворка Apache Spark

### Часть 1

В данной части работы рассмотрены:
* загрузка данных из HDFS;
* базовые преобразования данных;
* загрузка преобразованных данных в таблицу `Apache Airflow`.

Подключим необходимые библиотеки.

In [1]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    regexp_replace,
    regexp_extract_all,
    col,
    lit
)

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [2]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 1 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [3]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

:: loading settings :: url = jar:file:/opt/spark-3.5.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/user6/.ivy2/cache
The jars for the packages stored in: /home/user6/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7d1a541b-8250-4bbf-b172-8cbf373e4477;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0 in central
:: resolution report :: resolve 586ms :: artifacts dl 23ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spa

Для исследования будем использовать датасет `"US Used cars dataset"`, расположенный на платформе `Kaggle` по адресу https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset.

Датасет включает в себя информацию о более чем трех миллионах используемых машин в США. Он разрешен для использования в учебных целях.

Данный датасет уже загружен в `HDFS` по адресу: `hdfs:///datasets/used_cars_data.csv`

Указываем путь в `HDFS` для файла с данными.

In [7]:
path = "hdfs:///user/user6/epc_cut.csv"

Заполняем датафрейм данными из файла.

In [8]:
df = (spark.read.format("csv")
      .option("header", "true")
      .load(path)
)

Выводим фрагмент датафрейма на экран.

In [9]:
df.show()

25/01/14 19:44:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+--------------------+--------------------+--------------+--------+--------+-------------------------+---------------------+-----------------------+-------------------------+---------------------------+-------------+-------------+---------------+---------------+------------+------+--------------+--------------------+--------------------------+----------------------------+--------------------------+----------------------------+---------------------+-----------------------------+-----------------------+---------------------+-----------------------+--------------------+----------------------+----------------------+------------------------+----------------+-------------+--------------+-----------+---------------+-----------------+---------------------+----------------------+--------------------+-----------+---------------+----------------------+-------------------+-------------------+----------------------+--------------------+--------------------+-----------------+--------------------

In [10]:
df = df.select(
    "_c0", "LMK_KEY", "ADDRESS", "ADDRESS2", "ADDRESS3", "CURRENT_ENERGY_EFFICIENCY", "PROPERTY_TYPE", "INSPECTION_DATE",
    "HEATING_COST_CURRENT", "HOT_WATER_COST_CURRENT", "TOTAL_FLOOR_AREA", "NUMBER_HABITABLE_ROOMS",
    "NUMBER_HEATED_ROOMS"
)

Выведем на экран метаданные датасета.

In [11]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- LMK_KEY: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CURRENT_ENERGY_EFFICIENCY: string (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- INSPECTION_DATE: string (nullable = true)
 |-- HEATING_COST_CURRENT: string (nullable = true)
 |-- HOT_WATER_COST_CURRENT: string (nullable = true)
 |-- TOTAL_FLOOR_AREA: string (nullable = true)
 |-- NUMBER_HABITABLE_ROOMS: string (nullable = true)
 |-- NUMBER_HEATED_ROOMS: string (nullable = true)



In [12]:
df.show()

+---+--------------------+--------------------+-------------------------+-------------+---------------+--------------------+----------------------+----------------+----------------------+-------------------+
|_c0|             LMK_KEY|             ADDRESS|CURRENT_ENERGY_EFFICIENCY|PROPERTY_TYPE|INSPECTION_DATE|HEATING_COST_CURRENT|HOT_WATER_COST_CURRENT|TOTAL_FLOOR_AREA|NUMBER_HABITABLE_ROOMS|NUMBER_HEATED_ROOMS|
+---+--------------------+--------------------+-------------------------+-------------+---------------+--------------------+----------------------+----------------+----------------------+-------------------+
|  0|0003352408dad177e...|21 HERIOT GRANGE,...|                       53|         Flat|     2021-08-03|               119.0|                 583.0|            36.0|                   2.0|                2.0|
|  1|15149389392220170...| 9, Sea View Terrace|                       66|        House|     2017-01-26|               666.0|                 107.0|            88.0|    

Видно, что все столбцы датасета содержат строковый тип данных, что не соответствует ожиданиям. Выполним преобразования типов данных некоторых столбцов.

In [13]:
def transform_dataframe(data: DataFrame) -> DataFrame:
    """
    Преобразует столбцы DataFrame в указанные типы данных и
    выполняет необходимые преобразования.

    Args:
        data (DataFrame): Исходный DataFrame.

    Returns:
        DataFrame: Преобразованный DataFrame.
    """
    # Преобразуем столбцы в соответствующие типы данных
    data = data.withColumn("_c0",
                           col("_c0").cast("Integer"))
    data = data.withColumn("CURRENT_ENERGY_EFFICIENCY",
                           col("CURRENT_ENERGY_EFFICIENCY").cast("Integer"))
    data = data.withColumn("INSPECTION_DATE",
                           col("INSPECTION_DATE").cast("Date"))
    data = data.withColumn("HEATING_COST_CURRENT",
                           col("HEATING_COST_CURRENT").cast("Float"))
    data = data.withColumn("HOT_WATER_COST_CURRENT",
                           col("HOT_WATER_COST_CURRENT").cast("Float"))
    data = data.withColumn("TOTAL_FLOOR_AREA",
                           col("TOTAL_FLOOR_AREA").cast("Float"))
    data = data.withColumn("NUMBER_HABITABLE_ROOMS",
                           col("NUMBER_HABITABLE_ROOMS").cast("Float"))
    data = data.withColumn("NUMBER_HEATED_ROOMS",
                           col("NUMBER_HEATED_ROOMS").cast("Float"))
    

    return data

In [14]:
df = transform_dataframe(df)

In [15]:
df.show()

+---+--------------------+--------------------+-------------------------+-------------+---------------+--------------------+----------------------+----------------+----------------------+-------------------+
|_c0|             LMK_KEY|             ADDRESS|CURRENT_ENERGY_EFFICIENCY|PROPERTY_TYPE|INSPECTION_DATE|HEATING_COST_CURRENT|HOT_WATER_COST_CURRENT|TOTAL_FLOOR_AREA|NUMBER_HABITABLE_ROOMS|NUMBER_HEATED_ROOMS|
+---+--------------------+--------------------+-------------------------+-------------+---------------+--------------------+----------------------+----------------+----------------------+-------------------+
|  0|0003352408dad177e...|21 HERIOT GRANGE,...|                       53|         Flat|     2021-08-03|               119.0|                 583.0|            36.0|                   2.0|                2.0|
|  1|15149389392220170...| 9, Sea View Terrace|                       66|        House|     2017-01-26|               666.0|                 107.0|            88.0|    

In [22]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- LMK_KEY: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CURRENT_ENERGY_EFFICIENCY: integer (nullable = true)
 |-- PROPERTY_TYPE: string (nullable = true)
 |-- INSPECTION_DATE: date (nullable = true)
 |-- HEATING_COST_CURRENT: float (nullable = true)
 |-- HOT_WATER_COST_CURRENT: float (nullable = true)
 |-- TOTAL_FLOOR_AREA: float (nullable = true)
 |-- NUMBER_HABITABLE_ROOMS: float (nullable = true)
 |-- NUMBER_HEATED_ROOMS: float (nullable = true)



Видно, что теперь столбцы датафрейма содержат значения корректных типов.

Полученный датафрейм сохраним для дальнейшего использования. Сохранение выполним в таблицу `Apache Iceberg`. 

`Apache Iceberg` — это поддерживающий высокую производительность табличный формат для больших данных.

Сначала создадим базу данных, в которой будет расположена таблица. Во избежание путаницы, **каждую таблицу следует называть с использованием фамилии студента**.

In [23]:
database_name = "Efremenkov_database"

Создадим инструкцию SQL для добавления базы данных в каталог `Apache Spark`.

In [24]:
create_database_sql = f"""
CREATE DATABASE IF NOT EXISTS spark_catalog.{database_name}
"""

In [25]:
spark.sql(create_database_sql)

DataFrame[]

Установим созданную базу данных как текущую.

In [26]:
spark.catalog.setCurrentDatabase(database_name)

И, наконец, записываем преобразованный датафрейм в таблицу `sobd_lab1_table`.

In [27]:
# Сохранение DataFrame в виде таблицы
df.writeTo("sobd_lab1_table").using("iceberg").create()

После успешной записи можно посмотреть, какие таблицы входят в базу данных.

In [28]:
for table in spark.catalog.listTables():
    print(table.name)

sobd_lab1_table


Обратите внимание, что при необходимости созданные базу данных и таблицу можно удалить следующими командами.

In [ ]:
#spark.sql("DROP TABLE spark_catalog.Efremenkov_database.sobd_lab1_table")
#spark.sql("DROP DATABASE spark_catalog.Efremenkov_database")

DataFrame[]

После успешной записи таблицы останавливаем сессию `Apache Spark`.

In [ ]:
spark.stop()